In [1]:
import  functions.s3_fun as sf
import  functions.db_fun as db
import psycopg2
import psycopg2.extras
#from sqlalchemy import create_engine
import os

In [10]:

tabla='departments'

registros=sf.leer_csv_s3(tabla)
#db.insertar_registros(registros,tabla)
#registros.columns=['id', 'nombre', 'fecha_contrato', 'department_id', 'job_id']



In [21]:
registros_limpio=(registros[registros['id'].notna() & registros['department'].notna()]).head(200)

In [22]:
registros_limpio

,id,department
0,1,Product Management
1,2,Sales
2,3,Research and Development
3,4,Business Development
4,5,Engineering
5,6,Human Resources
6,7,Services
7,8,Support
8,9,Marketing
9,10,Training


In [5]:
registros_limpio

,id,department
0,1,Product Management
1,2,Sales
2,3,Research and Development
3,4,Business Development
4,5,Engineering
5,6,Human Resources
6,7,Services
7,8,Support
8,9,Marketing
9,10,Training


In [4]:

def conectar_db():
    try:
        connection = psycopg2.connect(
            user=os.environ.get('db_user'),
            password="roro1995.",
            host="db-test.ccgtzvdbixsj.us-east-2.rds.amazonaws.com",
            port="5432",
            database="postgres"
        )
        cursor = connection.cursor()
        return cursor,connection
        print("Connected to the database successfully")
    except (Exception, psycopg2.Error) as error:
        print("Error while connecting to PostgreSQL", error)

In [5]:
cur,conn=conectar_db()

In [2]:
cpntr=os.environ.get('db_user')

In [3]:
cpntr

'postgres'

In [37]:

if len(registros_limpio)>0:
    df_columns=list(registros_limpio)

    table='departments'
    columns=",".join(df_columns)
    values="VALUES ({})".format(",".join(["%s" for _ in df_columns]))
    
    cur.execute("SET SESSION TIME ZONE 'America/Lima'")
    insert_stmt="INSERT INTO {} ({}) {}".format(table,columns,values)
    cur.execute("TRUNCATE TABLE " + table + ";")
    psycopg2.extras.execute_batch(cur,insert_stmt,registros_limpio.values)
    conn.commit()

In [16]:
insert_stmt

'INSERT INTO hired_employees (id,nombre,fecha_contrato,department_id,job_id) VALUES (%s,%s,%s,%s,%s)'

In [35]:
conn.close()

In [45]:
def insertar_registros(datos,tabla):
    cur,conn=conectar_db()

    ##LIMPIEZA DE DATOS
    if tabla=='departments':
        datos_limpio=datos[datos['id'].notna() & datos['department'].notna()]
    
    if tabla=='employees':
        datos_limpio=datos[datos['job_id'].notna() & datos['department_id'].notna() & datos['nombre'].notna() & datos['fecha_contrato'].notna()]

    if tabla=='jobs':
        datos_limpio=datos[datos['id'].notna() & datos['job'].notna()]

    ##INSERTAMOS LOS DATOS LIMPIOS
    try:
        df_columns=list(datos_limpio)
        columns=",".join(df_columns)
        values="VALUES ({})".format(",".join(["%s" for _ in df_columns]))
        insert_stmt="INSERT INTO {} ({}) {}".format(tabla,columns,values)
        cur.execute("TRUNCATE TABLE " + tabla + ";")
        cur.execute("SET SESSION TIME ZONE 'America/Lima'")
        psycopg2.extras.execute_batch(cur,insert_stmt,datos_limpio.values)
        conn.commit()
        print('REGISTROS INSERTADOS CORRECTAMENTE')
        conn.close()
        print('CERRANDO CONEXION')
    except:
        print('ERROR AL INSERTAR LOS REGISTROS')


In [46]:
registros=sf.leer_csv_s3('departments')


In [43]:
registros

,id,department
0,1,Product Management
1,2,Sales
2,3,Research and Development
3,4,Business Development
4,5,Engineering
5,6,Human Resources
6,7,Services
7,8,Support
8,9,Marketing
9,10,Training


In [47]:
insertar_registros(registros,'departments')

REGISTROS INSERTADOS CORRECTAMENTE
CERRANDO CONEXION
